# Applying the MiniPile Pipeline to RefinedWeb

**Objectives:**

All of this is optional as per the original task proposal.<br>
But its fun!

- [x] Analyze the RefinedWeb dataset
- [.] Adapting SuperMiniPile pipeline for RefinedWeb, aiming for creating an equally performant yet smaller dataset (MiniRefinedWeb)
- [] Train Pythia $160\text{M}$ on RefinedWeb and MiniRefinedWeb, evaluate on MMLU and ARC-Challenge
- [] Train Pythia $1.4\text{B}$ with MiniRefinedWeb, evaluate pipeline performance on the MMLU and ARC benchmarks

---

## Analyze the RefinedWeb dataset

Other than The Pile, RefinedWeb is not as clearly structurally assembled.<br>
Where The Pile is made up from combining varying datasubsets, RefinedWeb is a collection of web pages that have been refined by:
- discarding irrelevant pages 
- applying a set of heuristics to remove low-quality content and 
- applying deduplication techniques to further remove near-duplicate pages

RefinedWeb consists of $968,000,015$ rows, amounting to $1.68\text{TB}$ of data.<br>
Each entry consists of the following columns:
- `content`: The textually representable content of the page
- `url`: The URL of the page
- `timestamp`: The timestamp of the page's last update
- `dump`: Which dump the page was found in (referring to CommonCrawl dumps)
- `segment`: The segment of the dump this page was found in
- `image_urls`: URLs of images found on this page

For our purposes, we will focus on the `content` column.<br>
We have to assume content is as varied as the web, making it hard to cluster or categorize.<br>
The key insight from MiniPile is that using semantic embeddings to express document relationships at some cluster resolution.<br>
RefinedWeb, with its 968 million web pages, requires a methodical approach to determine an optimal number of clusters. At best, said method should be applicable to other unstructured, maybe even larger datasets.

## The $k\text{atch}$

Approaching the issue from first principles, in order to find a reasonable $k$, I propose this staged approach:
- Randomly sample $n$ document indices from RefinedWeb, where $n$ is a **representative** fraction (say $1.5\%$ to $2\%$) of the dataset
- On this subset, perform silhouette analysis for $k$ in a range of $[100, 500]$ (or higher and lower, this is me winging it) in steps of $50$
- Plot the silhouette scores and determine the optimal $k$ based on the elbow method
- Use this $k$ to cluster the entire (then embedded) dataset
- Continue with the MiniPile pipeline from there.

## Ditching $k$-Means

Taking a step back, we need some form of preprocessing step to find $k$ for MiniPile's k-Means clustering. Why not drop k-Means altogether and use a more flexible clustering algorithm? I have to admit, I got thoroughly scared from my attempt at using HDBSCAN on The Pile earlier on. 
That attempt would lift the need for $k$, but HDBSCAN can't be batched, thus it will scale horribly.

However, ditching the need for a $k$ would allow us to more reasonable generalize MiniPile's pipeline across datasets, where we don't even need to know their content structure beforehand.<br>
I consulted literature, looking ideally for a large-scale- and high-dimensional-applicable clustering algorithm.

Specifically, I found the following papers (not all of them are related to the problem at hand, but they are interesting nonetheless):
- [Using Projection-Based Clustering to Find Distance- and Density-Based Clusters in High-Dimensional Data (Thrun, M. C. & Ultsch, Alfred. 2020)](https://link.springer.com/article/10.1007/s00357-020-09373-2)
- [An Efficient Density-based Clustering Algorithm for Higher-Dimensional Data (Boonchoo, et al. 2018)](https://arxiv.org/pdf/1801.06965)
- [Swarm Intelligence for Self-Organized Clustering (Thrun, M. C. & Ultsch, Afred. 2021)](https://arxiv.org/abs/2106.05521)
- [DPM: Fast and scalable clustering algorithm for large scale high dimensional datasets (Ghanem, et al. 2014)](https://ieeexplore.ieee.org/document/7050427)
- [K-DBSCAN: Identifying Spatial Clusters with Differing Density Levels](https://ieeexplore.ieee.org/document/7544972/)
- [FINDIT: a fast and intelligent subspace clustering algorithm using dimension voting (Woo, et al. 2004)](https://www.sciencedirect.com/science/article/abs/pii/S0950584903001411?via%3Dihub)
- [TeraHAC: Hierarchical Agglomerative Clustering of Trillion-Edge Graphs (Dhulipala, et al. 2023)](https://arxiv.org/abs/2308.03578)
- [A simple rapid sample-based clustering for large-scale data (Chen, et al. 2024)](https://www.sciencedirect.com/science/article/abs/pii/S0952197624007097)

### $k$-less Clustering with PCA'd BIRCH

An algorithm that by design could work with and cluster larger datasets without the need for a $k$ would be the Balanced Iterative Reducing and Clustering using Hierarchies (BIRCH) algorithm.

Papers related to this approach:
- [Efficient Clustering of High-Dimensional Data Sets with Application to Reference Matching (McCallum, et al. 2000)](https://pubs.dbs.uni-leipzig.de/dc/files/McCallum2000Efficientclusteringofhighdimensionaldatasetswith.pdf)
- [Improve BIRCH algorithm for big data clustering (Ramadhani, et al. 2020)](https://iopscience.iop.org/article/10.1088/1757-899X/725/1/012090)
- [Using Projection-Based Clustering to Find Distance- and Density-Based Clusters in High-Dimensional Data (Thrun, M. C. & Ultsch, Alfred. 2020)](https://link.springer.com/article/10.1007/s00357-020-09373-2)
- [Surpassing Cosine Similarity for Multidimensional Comparisons: Dimension Insensitive Euclidean Metric (DIEM) (Tessari, et al. 2024)](https://arxiv.org/abs/2407.08623)

We would ditch $k$-Means to instead, incrementally, build the Clustering Feature Tree, which then merges subclusters hierarchically based on density thresholds.<br>
Except for this tree structure being constantly in memory, we could now also feed the data into BIRCH in batched fashion.

To further accelerate the clustering process and compact both the 768-dimensional embeddings and thus the tree during clustering, we could apply Principal Component Analysis (PCA) before forwarding embeddings into BIRCH, effectively applying the findings of [(Ramadhani, et al. 2020)](https://iopscience.iop.org/article/10.1088/1757-899X/725/1/012090) and [(Thrun, M. C. & Ultsch, Alfred. 2020)](https://link.springer.com/article/10.1007/s00357-020-09373-2).

In this context, an issue that I wondered about with the original MiniPile approach was the employing of cosine similarity as measure for differentiation for $768$-dimensional embeddings during the $k$-Means clustering. The higher the dimensionality, the less expressiveness/nuance we can attain from metrics like cosine distance, as also discussed by [(Tessari, et al. 2024)](https://arxiv.org/abs/2407.08623). Therefore, applying PCA could help increase the effectiveness of cosine similarity.<br>
Furthermore, a lowered dimensionality for clustering could help prevent excessive splits in the CF Tree caused by (potentially more often in high dimensionality occuring) sparsities in data, leading to a more compacted tree and less memory usage.<br>
This would make the approach more scalable.

Structurally similar to how we approached $k$-Means, the steps for the pipeline would then be, processing the embedded shards incrementally:
- Cluster-Tree Emergence:
    - Apply PCA to the embeddings, reducing the dimensionality to $n$,
    - Feed the PCA-transformed embeddings into BIRCH, make that tree grow with checkpoints every $m$ shards (that is, save a checkpoint of the tree),
    - (We could again read an ominous `End_Here` flag set by the embedding step, to stop clustering (we could run off-set with embedding again))
- Clustering:
    - Assign all embeddings to the final tree,
    - Merge subclusters based on density thresholds/cosine similarity.

For a start, the E5-base-4k embedding step was realized with `05_embed_refinedweb_turbo.py`.<br>
I also experimented with a more distributed (across machines) approach to the embedding step, which would subsection the dataset into shard spaces covered per each running instance, and I used argparse for that in `05_embed_refinedweb_sectioned.py`, but I didn't employ that in the end.

---

Right here is where I had to cut it.<br>
Active embedding efforts on RefinedWeb began around christmas 2024 with as much resources I could muster, but that didn't suffice to get the job done in time.<br>
I also diverted larger, temporarily available resource pockets to the investigation of ablation studies on MiniPile, specifically the tiny, nano and pico versions.<br>
Still, I believe the theoretical groundwork layed out for MiniRefinedWeb could as-is be picked up to be realized later.